
# TODO
1.   Change location of importint data
2.   Methods comments
3.   Data requirements check
4.   assertions for sanity checks
5.   implement sample_size
6.   Custom vocabulary method


In [2]:
import keras.backend as K
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import re
import spacy
import spacy.cli
import time
from numpy import loadtxt
from keras.layers import Input, Embedding, LSTM, Lambda
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm
spacy.cli.download("en_core_web_lg")

In [13]:
sample_training_data = pd.read_csv('/content/quora.csv', engine='python',
                          usecols=['question1', 'question2', 'is_duplicate'],
                        skiprows=random.sample(range(1, 404290), 404290-10000))
lstm = ManhatanLSTM(sample_training_data)
print(lstm.training_data.shape)
lstm.training_data.head()

(10000, 3)


,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the best travel website in spain?,What is the best travel website?,0
2,Where can I find a European family office data...,Where do I find a U.S. family office database?,0
3,Which is the best gaming laptop under 60k INR?,Which is the best gaming laptop under Rs 60000?,1
4,Why do we cry when we are happy and when we ar...,Why do we cry?,0


In [14]:
lstm.create_vocabulary()
lstm.training_data.head()

100%|██████████| 10000/10000 [00:00<00:00, 25667.47it/s]


,question1,question2,is_duplicate,question1_vect,question2_vect
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,0,"[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 11, 9, 12]","[1, 2, 3, 4, 5, 4, 6, 7, 8, 9, 10, 11]"
1,what is the best travel website in spain,what is the best travel website,0,"[1, 2, 3, 13, 14, 15, 9, 16]","[1, 2, 3, 13, 14, 15]"
2,where can i find a european family office data...,where do i find a us family office database,0,"[17, 18, 19, 20, 21, 22, 23, 24, 25]","[17, 33, 19, 20, 21, 436, 23, 24, 25]"
3,which is the best gaming laptop under k inr,which is the best gaming laptop under rs,1,"[26, 2, 3, 13, 27, 28, 29, 30, 31]","[26, 2, 3, 13, 27, 28, 29, 1243, 50]"
4,why do we cry when we are happy and when we ar...,why do we cry,0,"[32, 33, 34, 35, 36, 34, 37, 38, 39, 36, 34, 3...","[32, 33, 34, 35]"


In [15]:
print(lstm.training_data.shape)
print(lstm.vocabulary)
print(len(lstm.vocabulary))

(10000, 5)
{'what': 1, 'is': 2, 'the': 3, 'step': 4, 'by': 5, 'guide': 6, 'to': 7, 'invest': 8, 'in': 9, 'share': 10, 'market': 11, 'india': 12, 'best': 13, 'travel': 14, 'website': 15, 'spain': 16, 'where': 17, 'can': 18, 'i': 19, 'find': 20, 'a': 21, 'european': 22, 'family': 23, 'office': 24, 'database': 25, 'which': 26, 'gaming': 27, 'laptop': 28, 'under': 29, 'k': 30, 'inr': 31, 'why': 32, 'do': 33, 'we': 34, 'cry': 35, 'when': 36, 'are': 37, 'happy': 38, 'and': 39, 'sad': 40, 'strongest': 41, 'structure': 42, 'or': 43, 'shape': 44, 'compression': 45, 'how': 46, 'many': 47, 'ways': 48, 'distribute': 49, '': 50, 'identical': 51, 'looking': 52, 'pencils': 53, 'students': 54, 'so': 55, 'that': 56, 'each': 57, 'student': 58, 'gets': 59, 'at': 60, 'least': 61, 'one': 62, 'pencil': 63, 'difference': 64, 'between': 65, 'gross': 66, 'salary': 67, 'ctc': 68, 'if': 69, 'hillary': 70, 'clinton': 71, 'could': 72, 'not': 73, 'continue': 74, 'her': 75, 'presidential': 76, 'campaign': 77, 'would

In [17]:
lstm.create_embeddings()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


100%|██████████| 15365/15365 [01:59<00:00, 129.09it/s]


In [24]:
lstm.embeddings.shape

(15366, 300)

In [12]:
class ManhatanLSTM:

    def __init__ (self, training_data):
        self.training_data = training_data
        # self.data = None
        # self.original_col = original_col
        # self.compare_text = compare_text
        # self.vocabulary = None
        # self.embeddings = None
        # self.model = None
        # self.X_train = None
        # self.y_train = None
        # self.X_val = None
        # self.y_val = None
        # self.model_history = None
        # self.model_load = None

        # self.create_vocabulary()
        # self.create_embeddings()

                                                # def load_training_data(self, training_data):
                                                #     ''' Load sample of quara training data '''

                                                #     self.training_data = training_data
                                                #     # print(f'data shape is {self.data.shape}')


    def text_to_word_list(self, text):
        ''' Preprocess text and keep it simple for practicality'''

        text = str(text).lower()
        text = re.sub('[^a-z ]', '', text)
        text = text

        return text


    def create_vocabulary(self):
        self.vocabulary = {}

        # '<unk>' will never be used, its only a placeholder for the [0, 0, ....0] embedding
        inverse_vocabulary = ['<unk>']  

        # Iterate through each sequence
        for sequence in [lstm.training_data.columns[0], lstm.training_data.columns[1]]:    

            # Duplicate sequence columns that will be transformed into vectors
            self.training_data[sequence+'_vect'] = self.training_data[sequence]

            # Iterate through each row
            for index in tqdm(range(len(self.training_data))):

                # Preprocess sequence text
                text_old = self.training_data.loc[index, sequence]
                text_new = self.text_to_word_list(text_old)
                self.training_data.at[index, sequence] = text_new

                word2id = []

                # Iterate through each word in the sequence
                for word in text_new.split(" "):

                    # Create id for word if its not in vocabulary yet
                    # Add word to text vector
                    if word not in self.vocabulary:
                        self.vocabulary[word] = len(inverse_vocabulary)
                        word2id.append(self.vocabulary[word])
                        inverse_vocabulary.append(word)
                    else:
                        word2id.append(self.vocabulary[word])

                self.training_data.at[index, sequence+'_vect'] = word2id

    def create_embeddings(self):
        nlp = spacy.load("en_core_web_lg")

        # Create embeddings matrix based on vocabulary
        embedding_dim = 300
        self.embeddings = np.zeros((len(self.vocabulary.items()) + 1, embedding_dim))

        for word, index in tqdm(self.vocabulary.items()):
            self.embeddings[index] = nlp(word).vector 

In [92]:

    # def create_train_validation(self):
    #     # Create training and validation data
    #     train, validation = train_test_split(self.data, test_size=0.2)

    #     # Create dictionaries with left/right keys for two sets of sequences 
    #     self.X_train = {'left':train['sequence1_vect'], 'right':train['sequence2_vect']}
    #     self.y_train = train['is_duplicate'].values

    #     self.X_val = {'left':validation['sequence1_vect'], 'right':validation['sequence2_vect']}
    #     self.y_val = validation['is_duplicate'].values

    #     # Calculate maximum length of sequences
    #     max_seq_length = max(train['sequence1_vect'].apply(lambda x: len(x)).max(),
    #                         train['sequence2_vect'].apply(lambda x: len(x)).max(),
    #                         validation['sequence1_vect'].apply(lambda x: len(x)).max(),
    #                         validation['sequence2_vect'].apply(lambda x: len(x)).max())
    #     print("max_seq_length:", max_seq_length)

    #     # Zero padding
    #     for self.data in [self.X_train, X_val]:
    #         for side in ['left', 'right']:
    #             self.data[side] = pad_sequences(self.data[side], maxlen=max_seq_length, padding='post')

    #     # Sanity check
    #     assert(self.X_train['left'].shape == self.X_train['right'].shape)
    #     assert(self.X_val['left'].shape == self.X_val['right'].shape)


    # def exponent_neg_manhattan_distance(left, right):
    #     ''' Calculates manhattan similarity estimate of the LSTMs outputs'''
    #     return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


    # def create_model():
    #     # Left and right input layers
    #     left_input, right_input = Input(shape=(max_seq_length,)), Input(shape=(max_seq_length,))

    #     # Embeddings layer
    #     embedding_layer = Embedding(input_dim=len(embeddings), # Size of vocabulary
    #                                 output_dim=embedding_dim, # Dimension of dense embedding
    #                                 weights=[embeddings], # ???
    #                                 trainable=False, #???
    #                                 input_length=max_seq_length # Length of input sequences
    #                                 )

    #     left_embedding, right_embedding = embedding_layer(left_input), embedding_layer(right_input)

    #     # LSTM later that is used on both sides
    #     lstm = LSTM(64)

    #     left_output, right_output = lstm(left_embedding), lstm(right_embedding)

    #     # Calculate manhattan distance for LSTM
    #     manhat_dist = Lambda(function=lambda x: self.exponent_neg_manhattan_distance(x[0], x[1]),
    #                         output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

    #     # Setup model
    #     self.model = Model(inputs=[left_input, right_input], outputs=[manhat_dist])


    # def compile_model():
    #     self.model.compile(loss='binary_crossentropy', optimizer='nadam', metrics='binary_accuracy')


    # def fit_model(batch_size=32, epochs=15):
    #     time_start = time.time()

    #     self.model_history = self.model.fit([self.X_train['left'], self.X_train['right']], self.y_train, 
    #                         batch_size=batch_size, 
    #                         epochs=epochs,
    #                         validation_data=([self.X_val['left'], self.X_val['right']], self.y_val))

    #     time_end = time.time()

    #     print(f'Training time is approximately {int((time_end - time_start) / 60)} minutes')


    # def plot_model_accuracy_and_loss():

    #     fig = go.Figure(data=go.Scatter(x=list(range(1,len(self.model_history.history['binary_accuracy'])+1)), 
    #                                     y=self.model_history.history['binary_accuracy'],
    #                                     name='Train'))
    #     fig.add_trace(go.Scatter(x=list(range(1,len(self.model_history.history['val_binary_accuracy'])+1)), 
    #                             y=self.model_history.history['val_binary_accuracy'],
    #                             name='Validation'))
    #     fig.update_layout(title_text='Model Accuracy',
    #                     title_x=0.5,
    #                     xaxis_title_text='Epoch',
    #                     yaxis_title_text='Accuracy',
    #                     height=400,
    #                     width=800,
    #                     plot_bgcolor ='#FFFFFF',
    #                     xaxis_linecolor='#000000',
    #                     xaxis_linewidth=2,
    #                     xaxis_mirror=True,
    #                     yaxis_linecolor='#000000',
    #                     yaxis_linewidth=2,
    #                     yaxis_mirror=True)

    #     fig2 = go.Figure(data=go.Scatter(x=list(range(1,len(self.model_history.history['loss'])+1)), 
    #                                     y=self.model_history.history['loss'],
    #                                     name='Train'))
    #     fig2.add_trace(go.Scatter(x=list(range(1,len(self.model_history.history['val_loss'])+1)), 
    #                             y=self.model_history.history['val_loss'],
    #                             name='Validation'))
    #     fig2.update_layout(title_text='Model Loss',
    #                     title_x=0.5,
    #                     xaxis_title_text='Epoch',
    #                     yaxis_title_text='Loss',
    #                     height=400,
    #                     width=800,
    #                     plot_bgcolor ='#FFFFFF',
    #                     xaxis_linecolor='#000000',
    #                     xaxis_mirror=True,
    #                     xaxis_linewidth=2,
    #                     yaxis_linecolor='#000000',
    #                     yaxis_mirror=True,
    #                     yaxis_linewidth=2,
    #     )

    #     fig.show()
    #     fig2.show()

    
    # def save_model(self):
    #     # save model and architecture to single file
    #     self.model.save("model.h5")
    #     print("Saved model to disk")


    # def load_model(self, path='/content/model.h5'):
    #     # load model
    #     self.model_load = load_model(path)


    # def summarize_model(self):
    #     # summarize model.
    #     self.model_load.summary()


    # def evaluate_model(self):
    #     # Evaluate the model
    #     score = self.model_load.evaluate([self.X_val['left'], self.X_val['right']], self.y_val, verbose=0)
    #     print(self.model_load.metrics_names[0], score[0])
    #     print("%s: %.2f%%" % (self.model_load.metrics_names[1], score[1]*100))

# TEST

In [ ]:
# ################################
# # Import leafly strains and concatinate each strain's feelings, helps, negatives and description into a single column

# strains = pd.read_json('/content/leafly.json')
# strains['strain_text'] = pd.Series(dtype='str')
# strains = strains.replace({None:""})

# columns = ['feeling_1', 'feeling_2', 'feeling_3', 'feeling_4', 'feeling_5',
#            'helps_1', 'helps_2', 'helps_3', 'helps_4', 'helps_5',
#            'negative_1', 'negative_2', 'negative_3', 'negative_4', 'negative_5',
#            'description']

# for i in range(0, len(strains)):
#     concat = ""
#     for col in columns:
#         concat = concat + " " + strains.loc[i, col]
#     strains.loc[i, 'strain_text'] = concat

# ################################

# '''
# This is an example of fake text submit by user in the website's UI.
# It is a concatination of user's selected feelings, helps and negatives terms.
# It also concatinates the user's submitted description that is written in free form.
# It is used to test the neural network model's predictive accuracy on marijuana strains.
# The text can be changed to test different scenarios.
# '''
# user_text = 'Headache Dizzy Nausea Happy Creative I am very sad every day feeling terrible and I need something that will cure my insomnia and anxiety and will help me get out of bed each day.'

# # Add user text to strains data
# print('user_text:', user_text)
# ################################

# strains[['strain', 'strain_text']].head()